In [3]:
#nltk.download_shell()

In [4]:
#nltk.download('wordnet')

In [5]:
#nltk.download('punkt')

In [6]:
import nltk
import pandas as pd
import collections
import itertools

In [8]:
ratings = pd.read_csv('Train.csv')

In [9]:
ratings.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


preprocessing

In [10]:
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from textblob import TextBlob

In [11]:
def Preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = [w for w in text.split(' ') if w not in stopwords.words('english')]
    text = [WordNetLemmatizer().lemmatize(token) for token in text]
    text = [WordNetLemmatizer().lemmatize(token,pos='v') for token in text]
    text = " ".join(text)
    return text

#ratings['text'] = ratings.text.apply(lambda x:Preprocessing(x))
#valid['text'] = valid.text.apply(lambda x:Preprocessing(x))
#test['text']= test.text.apply(lambda x:Preprocessing(x))
    
    #corpus['text'] = corpus['text'].replace('[A-Za-z0-9]+', ' ', regex=True)
    #corpus['text'] = corpus['text'].apply(lambda words: ''.join(word.lower() for word in words.split() if word not in stopwords.words('english')))
    #corpus['text'] = corpus['text'].apply(lambda x: str(TextBlob(x).correct()))
    
    #corpus = re.sub(r'[^\w\s]', '', corpus)
    #nopunc = [char for char in corpus if char not in string.punctuation]
    #nopunc = ''.join(nopunc)
    
    
    
    #return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    

In [12]:
small = ratings.head(5000)

In [13]:
small['text'] = small.text.apply(lambda x:Preprocessing(x))

C:\Users\HP\AppData\Local\Temp/ipykernel_8724/3242753296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['text'] = small.text.apply(lambda x:Preprocessing(x))


In [14]:
small.head()

,text,label
0,grow b 1965 watch love thunderbird mate school...,0
1,put movie dvd player sit coke chip expectation...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movie bore...,0
4,im die hard dad army fan nothing ever change g...,1


In [15]:
small['label'].value_counts()

0    2582
1    2418
Name: label, dtype: int64

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [17]:
tfidf_converter = TfidfVectorizer(tokenizer=Preprocessing)
classifier = LinearSVC()
X=small['text']
y=small['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline = Pipeline([
    ('tfidf',tfidf_converter),('cla',classifier)
])
pipeline.fit(X_train,y_train)

#features = tfidf_converter.fit_transform(small['text']).toarray()

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function Preprocessing at 0x000002004E39F550>)),
                ('cla', LinearSVC())])

In [18]:
y_predi = pipeline.predict(X_test)

In [19]:
confusion_matrix(y_test, y_predi)

array([[494, 250],
       [320, 436]], dtype=int64)

In [20]:
print(classification_report(y_test, y_predi))

              precision    recall  f1-score   support

           0       0.61      0.66      0.63       744
           1       0.64      0.58      0.60       756

    accuracy                           0.62      1500
   macro avg       0.62      0.62      0.62      1500
weighted avg       0.62      0.62      0.62      1500



In [56]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy_score(y_test, y_predi)

0.62

In [93]:
pipeline.predict(["the movie was boring and long"])

array([0], dtype=int64)

In [23]:
pipeline.predict(["I really enjoyed the movie"])

array([1], dtype=int64)

In [21]:
pipeline.predict(["the movie was not interesting and i don't like it"])

array([0], dtype=int64)

In [22]:
pipeline.predict(["the movie is not good"])

array([0], dtype=int64)

In [58]:
X_train.head()

1522    movie explore difficulty strain hope dream lov...
835     brief history time cosmological content docume...
358     im shock hat rat six seven still average seem ...
138     black candle rather muddle mess director bring...
299     many reason im fan fact base film filmmaker gi...
Name: text, dtype: object

In [59]:
from sklearn.naive_bayes import MultinomialNB

In [74]:
tfidf_converter = TfidfVectorizer(tokenizer=Preprocessing)
classifier = MultinomialNB()
X=small['text']
y=small['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = Pipeline([
    ('tfidf',tfidf_converter),('cla',classifier)
])
clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function Preprocessing at 0x00000132F1D80790>)),
                ('cla', MultinomialNB())])

In [75]:
y_pred = clf.predict(X_test)

In [76]:
confusion_matrix(y_test, y_pred)

array([[744,   0],
       [756,   0]], dtype=int64)

In [77]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.66       744
           1       0.00      0.00      0.00       756

    accuracy                           0.50      1500
   macro avg       0.25      0.50      0.33      1500
weighted avg       0.25      0.50      0.33      1500



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
import lightgbm as lgb

In [81]:
tfidf_converter = TfidfVectorizer(tokenizer=Preprocessing)
classifier = lgb.LGBMClassifier(max_depth=20,n_estimators=25,min_child_weight=0.0016,n_jobs=-1)
X=small['text']
y=small['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
pipeline3 = Pipeline([
    ('tfidf',tfidf_converter),('cla',classifier)
])
pipeline3.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function Preprocessing at 0x00000132F1D80790>)),
                ('cla',
                 LGBMClassifier(max_depth=20, min_child_weight=0.0016,
                                n_estimators=25))])

In [82]:
y_predict = clf.predict(X_test)

In [83]:
confusion_matrix(y_test, y_predict)

array([[795,   0],
       [704,   1]], dtype=int64)

In [85]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69       795
           1       1.00      0.00      0.00       705

    accuracy                           0.53      1500
   macro avg       0.77      0.50      0.35      1500
weighted avg       0.75      0.53      0.37      1500

